# **Menghubungkan dengan Directory Google Drive**

### **Mengimport Package yang dibutuhkan**

In [1]:
import pandas as pd
import xlrd as xl
import networkx as net
import matplotlib.pyplot as plt

# **Membuat Struktur Graph**

In [2]:
class Graph:
    # Menginisiasi graph yang akan digunakan
    def __init__(self, graph_dict=None, directed=True):
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            self.make_undirected()
    # Membuat undirected graph dengan menambahkan symmetric edges
    def make_undirected(self):
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.graph_dict.setdefault(b, {})[a] = dist
    # Menghubungkan dua verteks dan memberi weight, serta menambahkan inverse link jika graph undirected
    def connect(self, A, B, distance=1):
        self.graph_dict.setdefault(A, {})[B] = distance
        if not self.directed:
            self.graph_dict.setdefault(B, {})[A] = distance
    # Menampilkan semua node atau verteks yang bertetanggaan
    def get(self, a, b=None):
        links = self.graph_dict.setdefault(a, {})
        if b is None:
            return links
        else:
            return links.get(b)
    # Menampilkan semua verteks yang terdapat didalam graph
    def nodes(self):
        s1 = set([k for k in self.graph_dict.keys()])
        s2 = set([k2 for v in self.graph_dict.values() for k2, v2 in v.items()])
        nodes = s1.union(s2)
        return list(nodes)


# **Membuat Struktur Setiap Node**

In [3]:
class Node:
    # Menginisiasi node
    def __init__(self, name:str, parent:str):
        self.name = name
        self.parent = parent
        # Jarak ke start point
        self.g = 0 
        # Jarak ke goal point
        self.h = 0
        # Total Cost
        self.f = 0
    # Membandingkan nodes
    def __eq__(self, other):
        return self.name == other.name
    # Mengurutkan nodes
    def __lt__(self, other):
         return self.f < other.f
    # Menampilkan node
    def __repr__(self):
        return ('({0},{1})'.format(self.name, self.f))

# **Fungsi untuk A* Search**

In [4]:
def astar_search(graph, heuristics, start, end):

    open = []
    closed = []
    start_node = Node(start, None)
    goal_node = Node(end, None)
    open.append(start_node)
    
    while len(open) > 0:
        # Mengurutkan node yang berada di open list untuk mendapatkan node dengan cost terkecil di awal list
        open.sort()
        # Mengambil node dengan nilai cost terkecil
        current_node = open.pop(0)
        # Menambahkan current node tadi kedalam closed list
        closed.append(current_node)
        
        # Cek apakah goal sudah dicapai atau belum, kemudian return path nya
        if current_node == goal_node:
            path = []
            while current_node != start_node:
                path.append(current_node.name + ': ' + str(current_node.g))
                current_node = current_node.parent
            path.append(start_node.name + ': ' + str(start_node.g))
            # Menampilkan path yang telah di reversed
            return path[::-1]

        # Mengambil neighbors node dari current node
        neighbors = graph.get(current_node.name)
        
        # Loop neighbors
        for key, value in neighbors.items():
            # membuat  neighbor node
            neighbor = Node(key, current_node)
            # cek apakah neighbor ada di closed list atau tidak
            if(neighbor in closed):
                continue
            # Mengitung full path cost
            neighbor.g = current_node.g + graph.get(current_node.name, neighbor.name)
            neighbor.h = heuristics.get(neighbor.name)
            neighbor.f = neighbor.g + neighbor.h
            # cek apakah neighbor ada di open value atau tidak dan memiliki nilai f yang lebih kecil atau tidak
            if(add_to_open(open, neighbor) == True):
                open.append(neighbor)

    # Return None, jika tidak ada path yang memungkinkan
    return None

In [5]:
# Fungsi untuk mengecek apakah neighbor harus masuk ke openlist atau tidak
def add_to_open(open, neighbor):
    for node in open:
        if (neighbor == node and neighbor.f > node.f):
            return False
    return True

# **Mengaplikasikan A* Search**

In [7]:
def main():
    graph = Graph()
    
    df = pd.read_excel('Data Set.xlsx')
    # Masukkan data dalam graph
    for i in range(0,(len(df))):
       graph.connect(df.loc[i,'awal'], df.loc[i,'akhir'], df.loc[i,'jarak'])
    graph.make_undirected()
    # Menginput fungsi heuristics
    heuristics = {}
    hn = pd.read_excel('Data Set.xlsx', sheet_name='h(n)')
    for i in range(0,(len(hn))):
      heuristics[hn.loc[i,'node']] = hn.loc[i,'hn']

    # Menggunakan algoritma astar
    my = astar_search(graph, heuristics, 'Arus Jati', 'STMIK Nusa Mandiri')
    print(my)

# Menjalankan main function
if __name__ == "__main__": main()

['Arus Jati: 0', 'Jatinegara Kaum: 600', 'Jalan Taruna: 2200', 'Kayu Putih: 3800', 'Haji Ten: 5600', 'Percetakan Negara: 6250', 'Paseban Raya: 8350', 'Kramat: 9150', 'STMIK Nusa Mandiri: 10750']
